In [6]:
def findLowestModes(f, modeRange, nModes):
    start = time.perf_counter()

    r = np.arange(0,modeRange)
    xr,yr,zr=np.meshgrid(r,r,r)
    w=f(xr,yr,zr)
    sortedModes=np.argpartition(w.flatten(),nModes)[0:nModes]
    lowestModes=[]
    
    for k in sortedModes:
        idx = np.unravel_index(k,w.shape)
        freq=f(idx[0],idx[1],idx[2])
        if type(freq) is np.ndarray:
            lowestModes.append([idx, freq[0]])
        else:
            lowestModes.append([idx, freq])
    lowestModes.sort(key = lambda x:x[1])
    return lowestModes
   

### 13.1 - Golden Temple

A rectangular room is Ly = 20 m wide, Lx = 32.36 m deep, and Lz = 12.36 m high.

- Modes. Calculate the frequencies of the 27 lowest-frequency modes of the room. Tabulate the modes in ascending order of frequency (lowest to highest), indicating the mode numbers corresponding to each frequency.

- Modal excitation and detection. Assume the modes are excited by a volume velocity source located in a corner of the room. Indicate which of the 27 lowest-frequency modes listed above would be detected by a microphone placed exactly in the center of the room (i.e., x = Lx/2, y = Ly/2, and z = Lz/2).

### A:

In [13]:
import numpy as np
import time

n=100
nToSort=27
c=343
l=[32.36,20,12.36]

def f(x,y,z):
    return (c/2)*np.sqrt((x/[l[0]])**2+(y/[l[1]])**2+(z/[l[2]])**2)

findLowestModes(f,100,nToSort)

[[(0, 0, 0), 0.0],
 [(1, 0, 0), 5.299752781211372],
 [(0, 1, 0), 8.575000000000001],
 [(1, 1, 0), 10.080575605686299],
 [(2, 0, 0), 10.599505562422744],
 [(2, 1, 0), 13.633786824203712],
 [(0, 0, 1), 13.875404530744339],
 [(1, 0, 1), 14.853088245673293],
 [(3, 0, 0), 15.899258343634118],
 [(0, 1, 1), 16.311268371644204],
 [(0, 2, 0), 17.150000000000002],
 [(1, 1, 1), 17.150651749532965],
 [(2, 0, 1), 17.46070929428788],
 [(1, 2, 0), 17.95020555709482],
 [(2, 1, 1), 19.452686037142307],
 [(2, 2, 0), 20.161151211372598],
 [(0, 2, 1), 22.06013034621057],
 [(1, 2, 1), 22.6878101727284],
 [(2, 2, 1), 24.474453396544554],
 [(0, 3, 0), 25.724999999999998],
 [(1, 3, 0), 26.26524327970251],
 [(2, 3, 0), 27.823104484723316],
 [(0, 3, 1), 29.228453190201503],
 [(1, 3, 1), 29.705047642341164],
 [(0, 4, 0), 34.300000000000004],
 [(1, 4, 0), 34.70702204946368],
 [(0, 4, 1), 37.00022771405334]]

### B:

All modes except for the trivial mode (0,0,0) will be detected in the center of the room. 

### C:

$ \langle A \rangle = \sum{\alpha_i A_i}$

In [19]:
#Assume that the chairs approximately equal the surface area of the floor, since we aren't given dimensions
area={}
area['windows']=20*(3*4.85)
area['fg']=10*(2*4.85)
area['walls'] = 2*(l[1]*l[2]+l[0]*l[2])-area['fg']-area['windows']
area['ceil']=l[0]*l[1]
area['floor']=area['ceil']
area['seats']=area['floor']
#Absorption frequencies 
freqs=[125,250,500,1000,2000]

#Absorptions for all materials and frequency range
absorption = {}
absorption['windows']=[0.35,0.25,0.18,0.12,0.07]
absorption['walls']=[0.3,0.25,0.2,0.17,0.15]
absorption['fg']=[0.35,0.65,0.8,0.9,0.85]
absorption['ceil']=[0.07,0.17,0.4,0.55,0.65]
absorption['floor']=[0.02,0.06,0.14,0.35,0.6]
absorption['seats']=[0.2,0.35,0.55,0.65,0.6]
areaSum=0
for s in area.keys():
    areaSum+=area[s]

def absorbSum(freq):
    total=0
    areaSum=0
    nFreq=freqs.index(freq)
    for s in area.keys():
        total+=area[s]*absorption[s][nFreq]
    return total
#13.35
fS=343*np.sqrt(6/absorbSum(125))
print("fS = "+str(fS))

L=4*(l[0]+l[1]+l[2])
V=l[0]*l[1]*l[2]

freq=dFreq=fS/2
nModes=4*np.pi*freq**3/c**3+np.pi*areaSum*freq**2/(2*c**2)+L*freq/(8*c)
print("Modes below fS: " +str(nModes))

fS = 34.43254032098826
Modes below fS: 14.431741918127608


### D, E:


In [20]:
for f in freqs:
    sabine = 13.82*4*V/(c*absorbSum(f))
    eyring = 0.161*V/(-areaSum*np.log(1-absorbSum(f)/areaSum))
    print("Sabine T60 at frequency {0} = {1}".format(str(f),str(sabine)))
    print("Eyring T60 at frequency {0} = {1}".format(str(f),str(eyring)))
    

Sabine T60 at frequency 125 = 2.165357121738605
Eyring T60 at frequency 125 = 1.95738545219724
Sabine T60 at frequency 250 = 1.7474924894711659
Eyring T60 at frequency 250 = 1.5381183390806281
Sabine T60 at frequency 500 = 1.2680601280184631
Eyring T60 at frequency 500 = 1.0552735411902712
Sabine T60 at frequency 1000 = 1.0076390124513248
Eyring T60 at frequency 1000 = 0.7909817819696827
Sabine T60 at frequency 2000 = 0.8977366787721952
Eyring T60 at frequency 2000 = 0.6784658523374307


### F:

In [21]:
print("Critical distance at 500hz: "+str(0.25*np.sqrt(absorbSum(500)/np.pi)))

Critical distance at 500hz: 4.49739047333238


### G:

In [22]:
power = 2/1000
pref=20e-6
c=343
rho=1.225

p=np.sqrt(rho*c*4*power/absorbSum(500))
spl=20*np.log10(p/pref)
print(spl)

69.17269157843221


This seems reasonable - google says violins are about 78 dB on the low end

### H:

Surface absorbtion seems to outweight air attenuation

In [23]:
rh=50
for f in freqs:
    m=5.5e-4*(50/rh)*(f/1000)**1.7
    print(4*m*V/absorbSum(f))

0.0008618373694673516
0.0022597588666360123
0.0053276737357613345
0.013754782901515373
0.03981518413950064


### 13.2 - hot tub:

$p_1(r,\theta,z,t)=A*cos(k_l z)*J_m(k_{mn}r)cos(m\theta +\phi_{lmn})e^{i\omega_{lmn}t}$

For the fixed end, $\frac{dp_1}{dz}=0$. For the open end, $p_1=0$.c

At the fixed end, $\frac{dZ}{dz}=k_l sin(k_l z)$ - this is valid for any $k_l$ at z=0

At the open end, $z=h$, $k_l=\frac{\pi}{2h}+\frac{\pi l}{h}$ 

At the circular edges, $\frac{j_{m,n}}{r}=k_{mn}$

making the modal frequencies $f_{lmn}=\frac{c}{2}\sqrt{(\frac{\pi}{2h}+\frac{\pi l}{h})^2+(\frac{j_{m,n}}{r})^2}$

*I think the book notation is different regarding the directions of l,m, and n, but the math should still work out for the BCs*

In [24]:
from scipy.special import *
#jn_zeros(order, num roots) 
#jv(order, argument)

d=5
h=2

n=100
r = np.arange(0,n)
lr,mr,nr=np.meshgrid(r,r,r)
c=1500

def kl(l):
    return np.pi/(2*h)+np.pi*l/h
#Do all bessel calcs at the beginning to avoid repetition
#not sure if these are stored as value tables or calculated
bessel=np.zeros([n,n])
for m in range(n):
    bessel[m,:]=jn_zeros(int(m),n)
def kmn(m,n):
    return bessel[m,n-1]
def tubFreqs(l,m,n):
    return (c/2)*np.sqrt(kl(l)**2+kmn(m,n))
findLowestModes(tubFreqs, 100, 10)

[[(0, 0, 1), 1303.7226146422674],
 [(0, 1, 1), 1581.8700603929274],
 [(0, 0, 2), 1857.9618448078656],
 [(1, 0, 1), 2115.5422221586723],
 [(0, 0, 3), 2283.5718142283686],
 [(1, 0, 2), 2495.565758413957],
 [(2, 0, 1), 3166.5709165197636],
 [(2, 0, 2), 3432.127755515665],
 [(3, 0, 1), 4284.232729762467],
 [(4, 0, 1), 5427.5183126354805]]

### 13.7

$c_{gr}= c\sqrt{1-(\frac{\omega_{lm}}{\omega})^2}$

$2 \pi f_{co} = \omega_{co} = ck_{lm}$

$f_{co} = c\frac{\alpha_{mn}}{2\pi\alpha}$

In [25]:
jPrimeZeros=[0,3.83,7.02,10.17,13.32,16.47,19.61,22.76,25.90,29.04,32.18,35.33,38.47,41.61,44.76,47.9,51.04,54.18,57.32,60.46]
c=343

L=20
a=5.11/100
fBase = 5000

for alpha in jPrimeZeros:
    f=c*alpha/(2*np.pi*a)
    cgr=c*np.sqrt(1-(f/fBase)**2) #factor of 2pi comes out and leaves freqs
    t=2*L/cgr
    print(t)

0.11661807580174927
0.20289672440248577
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


<ipython-input-25-de3a1e06ee93>:10: RuntimeWarning: invalid value encountered in sqrt
  cgr=c*np.sqrt(1-(f/fBase)**2) #factor of 2pi comes out and leaves freqs


Everything after mode 2 seems to be imaginary, I'd hazard a guess that there is also an upper limit to number of modes in waveguides in addition to the cutoff frequency. The max difference in arrival time is 86ms, which is significant. 